<a href="https://colab.research.google.com/github/ViniciusARZ/Data-Structures-and-Algorithms/blob/main/Tranformation%20ETLs/Slang's_CAC_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## Gspread Usual Stack
!pip install -q --upgrade gspread
!pip install -q pandas
!pip install -q -U -q PyDrive
!pip install -q datetime
!pip install -q pendulum
!pip install -q matplotlib


import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
import datetime
import pendulum

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Costs

## Employees

In [3]:
#Importing from the Outreach Extras
sheets = gc.open_by_key('1_mwE9cKPmfzSBYUqbf1Ttzwcp5vuLDmYy0mjx3vHFQA')
sheets_data_weights = sheets.worksheet('Weight')
sheets_rows_weights = sheets_data_weights.get_all_values()
weights = pd.DataFrame.from_records(sheets_rows_weights[1:],columns=sheets_rows_weights[0])
weights = weights.replace('',0).astype({'Prospecting':'float','Outreach':'float','Demo':'float','Pilot':'float',
                                            'MQL':'float','SQL':'float','Opp.':'float','Deal':'float','B2B':'float','B2C':'float'})
weights['Start_Month'] = pd.to_datetime(weights['Start_Month'])
weights['End_Month'] = pd.to_datetime(weights['End_Month'])
weights.head(3)

,Start_Month,End_Month,Area,Title,Prospecting,Outreach,Demo,Pilot,MQL,SQL,Opp.,Deal,B2B,B2C
0,2018-01-01,2018-12-01,Business Development,BDR Manager,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2018-01-01,2018-12-01,Business Development,Business Development Intern,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2018-01-01,2018-12-01,Business Development,Business Development Representative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
sheets_data_values = sheets.worksheet('Values')
sheets_rows_values = sheets_data_values.get_all_values()
values = pd.DataFrame.from_records(sheets_rows_values[1:],columns=sheets_rows_values[0])
values['Date'] = pd.to_datetime(values['Date'])
values[values['Position']=='Head of Sales'].head(3)

,Bad Date,Country,Year,Month,Position,SALARY,Date,Area,Comissions
17,1/1/2020,COL,2020,Jan,Head of Sales,7649.48,2020-01-01,Sales,3645.96
44,1/1/2020,BRA,2020,Jan,Head of Sales,0.00,2020-01-01,Sales,0.00
71,1/1/2020,MEX,2020,Jan,Head of Sales,0.00,2020-01-01,Sales,0.00


In [5]:
values['SALARY'] = pd.to_numeric(values['SALARY'])
values['Comissions'] = pd.to_numeric(values['Comissions'])

In [6]:
merge = values.merge(weights, left_on = 'Position', right_on = 'Title', how='left')
merge = merge.loc[(merge['Date'] >= merge['Start_Month']) & (merge['Date'] < merge['End_Month'])].drop_duplicates()

merge['Salary + Comission'] = merge['SALARY'] + merge['Comissions'] 

#wc = weighted spend
merge['B2B WS'] = merge['Salary + Comission']*merge['B2B']
merge['B2C WS'] = merge['Salary + Comission']*merge['B2C']

merge['MQL WS'] = merge['Salary + Comission']*merge['MQL']
merge['SQL WS'] = merge['Salary + Comission']*merge['SQL']
merge['OPP WS'] = merge['Salary + Comission']*merge['Opp.']

merge['Opportunity Total WS'] = merge['MQL WS'] + merge['SQL WS'] + merge['OPP WS']
merge['Deal WS'] = merge['Salary + Comission']*merge['Deal']

merge['Marketing Cost'] = np.where(merge['Area_x']=='Marketing',merge['MQL WS']+merge['SQL WS']+merge['OPP WS']+merge['Deal WS']+merge['B2C WS']+merge['B2B WS'],0)

merge['Sales Cost'] = (merge['Opportunity Total WS'] + merge['Deal WS']) - merge['Marketing Cost']

In [7]:
merge['Month - Year'] = merge['Date'].dt.strftime('%Y-%m')

merge[['Month - Year','Salary + Comission','B2B WS','B2C WS','MQL WS',
        'SQL WS','OPP WS','Opportunity Total WS','Deal WS','Marketing Cost','Sales Cost']].groupby('Month - Year').sum().reset_index()

,Month - Year,Salary + Comission,B2B WS,B2C WS,MQL WS,SQL WS,OPP WS,Opportunity Total WS,Deal WS,Marketing Cost,Sales Cost
0,2020-01,66937.00,56649.9640,10287.0360,13072.450,19574.430,13716.0480,46362.9280,10287.0360,8185.96,48464.0040
1,2020-02,71202.81,59986.2240,11216.5860,13881.095,19933.095,14955.4480,48769.6380,11216.5860,9768.90,50217.3240
2,2020-03,75226.45,63529.5340,11696.9160,13901.615,22335.115,15595.8880,51832.6180,11696.9160,9140.28,54389.2540
3,2020-04,68211.54,55427.7420,12783.7980,9839.850,15759.030,17045.0640,42643.9440,12783.7980,6218.82,49208.9220
4,2020-05,46476.98,39970.0730,6506.9070,9468.630,15318.660,8675.8760,33463.1660,6506.9070,6937.10,33032.9730
5,2020-06,58603.99,48899.4250,9704.5650,11482.180,14773.260,12939.4200,39194.8600,9704.5650,7767.94,41131.4850
6,2020-07,59309.99,48235.7750,11074.2150,10430.970,11964.970,14765.6200,37161.5600,11074.2150,7467.98,40767.7950
7,2020-08,62233.14,52225.5870,10007.5530,14512.430,14362.200,13343.4040,42218.0340,10007.5530,10039.72,42185.8670
8,2020-09,59629.53,49856.3730,9773.1570,14260.210,12792.130,13030.8760,40083.2160,9773.1570,8140.24,41716.1330
9,2020-10,67988.92,55964.3560,12024.5640,12998.070,14908.970,16032.7520,43939.7920,12024.5640,8196.14,47768.2160


## Tools

In [23]:
sheets_data_weights_tools = sheets.worksheet('Weight Tools')
sheets_rows_weights_tools = sheets_data_weights_tools.get_all_values()
weights_tools = pd.DataFrame.from_records(sheets_rows_weights_tools[1:],columns=sheets_rows_weights_tools[0])
weights_tools = weights_tools.replace('',0).astype({'Prospecting':'float','Outreach':'float','Demo':'float','Pilot':'float',
                                            'MQL':'float','SQL':'float','Opp.':'float','Deal':'float','B2B':'float','B2C':'float'})
weights_tools['Start_Month'] = pd.to_datetime(weights_tools['Start_Month'])
weights_tools['End_Month'] = pd.to_datetime(weights_tools['End_Month'])
weights_tools.head(3)

,Start_Month,End_Month,Area,Tool,Prospecting,Outreach,Demo,Pilot,MQL,SQL,Opp.,Deal,B2B,B2C
0,2018-01-01,2018-12-01,Marketing,Google,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2018-01-01,2018-12-01,Marketing,Hubspot Marketing,0.33,0.33,0.33,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2018-01-01,2018-12-01,Business Development,Mailshake.com,1.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
sheets_data_values_tools = sheets.worksheet('Values Tools')
sheets_rows_values_tools = sheets_data_values_tools.get_all_values()
values_tools = pd.DataFrame.from_records(sheets_rows_values_tools[1:],columns=sheets_rows_values_tools[0])
values_tools['Date'] = pd.to_datetime(values_tools['Date'])
values_tools.head(3)

,Date,Area,Tool,Value
0,2018-01-01,Marketing,Google,1154.00
1,2018-01-01,Marketing,Hubspot Marketing,226.00
2,2018-01-01,Business Development,Mailshake.com,46.00


In [25]:
values_tools['Value'] = pd.to_numeric(values_tools['Value'])

merge_tools = values_tools.merge(weights_tools, left_on = 'Tool', right_on = 'Tool', how='left')
merge_tools = merge_tools.loc[(merge_tools['Date'] >= merge_tools['Start_Month']) & (merge_tools['Date'] < merge_tools['End_Month'])].drop_duplicates()

merge_tools['B2B WS'] = merge_tools['Value']*merge_tools['B2B']
merge_tools['B2C WS'] = merge_tools['Value']*merge_tools['B2C']

merge_tools['MQL WS'] = merge_tools['Value']*merge_tools['MQL']
merge_tools['SQL WS'] = merge_tools['Value']*merge_tools['SQL']
merge_tools['OPP WS'] = merge_tools['Value']*merge_tools['Opp.']

merge_tools['Opportunity Total WS'] = merge_tools['MQL WS'] + merge_tools['SQL WS'] + merge_tools['OPP WS']
merge_tools['Deal WS'] = merge_tools['Value']*merge_tools['Deal']

merge_tools['Marketing Cost'] = np.where(merge_tools['Area_x']=='Marketing',merge_tools['MQL WS']+merge_tools['SQL WS']+
                                          merge_tools['OPP WS']+merge_tools['Deal WS']+merge_tools['B2C WS']+merge_tools['B2B WS'],0)

merge_tools['Sales Cost'] = (merge_tools['Opportunity Total WS'] + merge_tools['Deal WS']) - merge_tools['Marketing Cost']

In [26]:
merge_tools['Month - Year'] = merge_tools['Date'].dt.strftime('%Y-%m')

merge_tools[['Month - Year','Value','B2B WS','B2C WS','MQL WS',
        'SQL WS','OPP WS','Opportunity Total WS','Deal WS','Marketing Cost','Sales Cost']].groupby('Month - Year').sum().reset_index()

,Month - Year,Value,B2B WS,B2C WS,MQL WS,SQL WS,OPP WS,Opportunity Total WS,Deal WS,Marketing Cost,Sales Cost
0,2018-01,2779.44,1605.44,1174.0,0.0000,0.0000,0.0000,0.0000,0.0000,1380.00,-1380.00
1,2018-02,3932.44,1412.44,2520.0,0.0000,0.0000,0.0000,0.0000,0.0000,2500.00,-2500.00
2,2018-03,2778.44,1258.44,1520.0,0.0000,0.0000,0.0000,0.0000,0.0000,1500.00,-1500.00
3,2018-04,2859.00,1339.00,1520.0,0.0000,0.0000,0.0000,0.0000,0.0000,1500.00,-1500.00
4,2018-05,4360.00,1340.00,3020.0,0.0000,0.0000,0.0000,0.0000,0.0000,3000.00,-3000.00
5,2018-06,11368.00,1848.00,9520.0,0.0000,0.0000,0.0000,0.0000,0.0000,9900.00,-9900.00
6,2018-07,8024.00,1504.00,6520.0,0.0000,0.0000,0.0000,0.0000,0.0000,6160.00,-6160.00
7,2018-08,5911.00,1391.00,4520.0,0.0000,0.0000,0.0000,0.0000,0.0000,4500.00,-4500.00
8,2018-09,4442.00,922.00,3520.0,0.0000,0.0000,0.0000,0.0000,0.0000,3500.00,-3500.00
9,2018-10,3869.00,849.00,3020.0,0.0000,0.0000,0.0000,0.0000,0.0000,3000.00,-3000.00


## Others

In [27]:
sheets_data_weights_others = sheets.worksheet('Weight Others')
sheets_rows_weights_others = sheets_data_weights_others.get_all_values()
weights_others = pd.DataFrame.from_records(sheets_rows_weights_others[1:],columns=sheets_rows_weights_others[0])
weights_others = weights_others.replace('',0).astype({'Prospecting':'float','Outreach':'float','Demo':'float','Pilot':'float',
                                            'MQL':'float','SQL':'float','Opp.':'float','Deal':'float','B2B':'float','B2C':'float'})
weights_others['Start_Month'] = pd.to_datetime(weights_others['Start_Month'])
weights_others['End_Month'] = pd.to_datetime(weights_others['End_Month'])
weights_others.head(3)

,Start_Month,End_Month,Area,Tool,Prospecting,Outreach,Demo,Pilot,MQL,SQL,Opp.,Deal,B2B,B2C
0,2018-01-01,2023-12-01,Marketing,Extra Marketing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,2018-01-01,2023-12-01,Sales,Travel Expenses,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2018-01-01,2023-12-01,Sales,Prof. and Outsourced Services,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [28]:
sheets_data_values_others = sheets.worksheet('Values Others')
sheets_rows_values_others = sheets_data_values_others.get_all_values()
values_others = pd.DataFrame.from_records(sheets_rows_values_others[1:],columns=sheets_rows_values_others[0])
values_others['Date'] = pd.to_datetime(values_others['Date'])
values_others.head(3)

,Date,Area,Tool,Value
0,2020-01-01,Sales,Travel Expenses,469.00
1,2020-02-01,Sales,Travel Expenses,1112.00
2,2020-03-01,Sales,Travel Expenses,149.00


In [29]:
values_others['Value'] = pd.to_numeric(values_others['Value'])

merge_others = values_others.merge(weights_others, left_on = 'Tool', right_on = 'Tool', how='left')
merge_others = merge_others.loc[(merge_others['Date'] >= merge_others['Start_Month']) & (merge_others['Date'] < merge_others['End_Month'])].drop_duplicates()

merge_others['B2B WS'] = merge_others['Value']*merge_others['B2B']
merge_others['B2C WS'] = merge_others['Value']*merge_others['B2C']

merge_others['MQL WS'] = merge_others['Value']*merge_others['MQL']
merge_others['SQL WS'] = merge_others['Value']*merge_others['SQL']
merge_others['OPP WS'] = merge_others['Value']*merge_others['Opp.']

merge_others['Opportunity Total WS'] = merge_others['MQL WS'] + merge_others['SQL WS'] + merge_others['OPP WS']
merge_others['Deal WS'] = merge_others['Value']*merge_others['Deal']

merge_others['Marketing Cost'] = np.where(merge_others['Area_x']=='Marketing',merge_others['MQL WS']+merge_others['SQL WS']+
                                          merge_others['OPP WS']+merge_others['Deal WS']+merge_others['B2C WS']+merge_others['B2B WS'],0)

merge_others['Sales Cost'] = (merge_others['Opportunity Total WS'] + merge_others['Deal WS']) - merge_others['Marketing Cost']

In [30]:
merge_others

,Date,Area_x,Tool,Value,Start_Month,End_Month,Area_y,Prospecting,Outreach,Demo,...,B2C,B2B WS,B2C WS,MQL WS,SQL WS,OPP WS,Opportunity Total WS,Deal WS,Marketing Cost,Sales Cost
0,2020-01-01,Sales,Travel Expenses,469.00,2018-01-01,2023-12-01,Sales,0.0,0.0,0.0,...,0.0,469.00,0.00,0.0,0.0,0.0,0.0,469.0,0.00,469.00
1,2020-02-01,Sales,Travel Expenses,1112.00,2018-01-01,2023-12-01,Sales,0.0,0.0,0.0,...,0.0,1112.00,0.00,0.0,0.0,0.0,0.0,1112.0,0.00,1112.00
2,2020-03-01,Sales,Travel Expenses,149.00,2018-01-01,2023-12-01,Sales,0.0,0.0,0.0,...,0.0,149.00,0.00,0.0,0.0,0.0,0.0,149.0,0.00,149.00
3,2020-04-01,Sales,Travel Expenses,-52.00,2018-01-01,2023-12-01,Sales,0.0,0.0,0.0,...,0.0,-52.00,-0.00,-0.0,-0.0,-0.0,-0.0,-52.0,0.00,-52.00
4,2020-05-01,Sales,Travel Expenses,5.00,2018-01-01,2023-12-01,Sales,0.0,0.0,0.0,...,0.0,5.00,0.00,0.0,0.0,0.0,0.0,5.0,0.00,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2022-05-01,Marketing,Extra Marketing,2156.00,2018-01-01,2023-12-01,Marketing,0.0,0.0,0.0,...,1.0,2156.00,2156.00,0.0,0.0,0.0,0.0,0.0,4312.00,-4312.00
95,2022-06-01,Marketing,Extra Marketing,2604.15,2018-01-01,2023-12-01,Marketing,0.0,0.0,0.0,...,1.0,2604.15,2604.15,0.0,0.0,0.0,0.0,0.0,5208.30,-5208.30
96,2022-07-01,Marketing,Extra Marketing,5911.00,2018-01-01,2023-12-01,Marketing,0.0,0.0,0.0,...,1.0,5911.00,5911.00,0.0,0.0,0.0,0.0,0.0,11822.00,-11822.00
97,2022-08-01,Marketing,Extra Marketing,4424.00,2018-01-01,2023-12-01,Marketing,0.0,0.0,0.0,...,1.0,4424.00,4424.00,0.0,0.0,0.0,0.0,0.0,8848.00,-8848.00


In [31]:
merge_others['Month - Year'] = merge_others['Date'].dt.strftime('%Y-%m')

merge_others[['Month - Year','Value','B2B WS','B2C WS','MQL WS',
        'SQL WS','OPP WS','Opportunity Total WS','Deal WS','Marketing Cost','Sales Cost']].groupby('Month - Year').sum().reset_index()

,Month - Year,Value,B2B WS,B2C WS,MQL WS,SQL WS,OPP WS,Opportunity Total WS,Deal WS,Marketing Cost,Sales Cost
0,2020-01,5684.00,5684.00,1343.00,0.0,0.0,0.0,0.0,4341.00,2686.00,1655.00
1,2020-02,6589.00,6589.00,1756.00,0.0,0.0,0.0,0.0,4833.00,3512.00,1321.00
2,2020-03,6161.95,6161.95,1177.95,0.0,0.0,0.0,0.0,4984.00,2355.90,2628.10
3,2020-04,3317.81,3317.81,0.00,0.0,0.0,0.0,0.0,3317.81,0.00,3317.81
4,2020-05,4873.19,4873.19,0.00,0.0,0.0,0.0,0.0,4873.19,0.00,4873.19
5,2020-06,2732.00,2732.00,523.00,0.0,0.0,0.0,0.0,2209.00,1046.00,1163.00
6,2020-07,2395.33,2395.33,0.00,0.0,0.0,0.0,0.0,2395.33,0.00,2395.33
7,2020-08,2512.00,2512.00,0.00,0.0,0.0,0.0,0.0,2512.00,0.00,2512.00
8,2020-09,3911.00,3911.00,0.00,0.0,0.0,0.0,0.0,3911.00,0.00,3911.00
9,2020-10,3283.00,3283.00,0.00,0.0,0.0,0.0,0.0,3283.00,0.00,3283.00


## Merge Final

In [32]:
merge.columns
t = merge[['Month - Year','Date','Position','Salary + Comission','MQL WS','SQL WS','OPP WS','Deal WS','B2B WS','B2C WS','Marketing Cost']].rename(columns={'Position':'Name','Salary + Comission':'Value $'})
t['Type'] = 'Employee'
t2 = merge_tools[['Month - Year','Date','Tool','Value','MQL WS','SQL WS','OPP WS','Deal WS','B2B WS','B2C WS','Marketing Cost']].rename(columns={'Tool':'Name','Value':'Value $'})
t2['Type'] = 'Tool'
t3 = merge_others[['Month - Year','Date','Tool','Value','MQL WS','SQL WS','OPP WS','Deal WS','B2B WS','B2C WS','Marketing Cost']].rename(columns={'Tool':'Name','Value':'Value $'})
t3['Type'] = 'Others'

t_m = pd.concat([t,t2,t3])

t_f = t_m.groupby('Month - Year').sum().reset_index()
t_f

,Month - Year,Value $,MQL WS,SQL WS,OPP WS,Deal WS,B2B WS,B2C WS,Marketing Cost
0,2018-01,2779.44,0.0000,0.0000,0.0000,0.0000,1605.4400,1174.0000,1380.00
1,2018-02,3932.44,0.0000,0.0000,0.0000,0.0000,1412.4400,2520.0000,2500.00
2,2018-03,2778.44,0.0000,0.0000,0.0000,0.0000,1258.4400,1520.0000,1500.00
3,2018-04,2859.00,0.0000,0.0000,0.0000,0.0000,1339.0000,1520.0000,1500.00
4,2018-05,4360.00,0.0000,0.0000,0.0000,0.0000,1340.0000,3020.0000,3000.00
5,2018-06,11368.00,0.0000,0.0000,0.0000,0.0000,1848.0000,9520.0000,9900.00
6,2018-07,8024.00,0.0000,0.0000,0.0000,0.0000,1504.0000,6520.0000,6160.00
7,2018-08,5911.00,0.0000,0.0000,0.0000,0.0000,1391.0000,4520.0000,4500.00
8,2018-09,4442.00,0.0000,0.0000,0.0000,0.0000,922.0000,3520.0000,3500.00
9,2018-10,3869.00,0.0000,0.0000,0.0000,0.0000,849.0000,3020.0000,3000.00


## Quarter

In [33]:
merge_tools['Quarter - Year'] = merge_tools['Date'].dt.year.astype(str) + ' ' + merge_tools['Date'].dt.quarter.astype(str)

merge['Quarter - Year'] = merge['Date'].dt.year.astype(str) + ' ' + merge['Date'].dt.quarter.astype(str)

merge_others['Quarter - Year'] = merge_others['Date'].dt.year.astype(str) + ' ' + merge_others['Date'].dt.quarter.astype(str)


t_q = merge[['Quarter - Year','Date','Position','Salary + Comission','MQL WS','SQL WS','OPP WS','Deal WS','B2B WS','B2C WS','Marketing Cost']].rename(columns={'Position':'Name','Salary + Comission':'Value $'})
t_q['Type'] = 'Employee'
t2_q = merge_tools[['Quarter - Year','Date','Tool','Value','MQL WS','SQL WS','OPP WS','Deal WS','B2B WS','B2C WS','Marketing Cost']].rename(columns={'Tool':'Name','Value':'Value $'})
t2_q['Type'] = 'Tool'

t3_q = merge_others[['Quarter - Year','Date','Tool','Value','MQL WS','SQL WS','OPP WS','Deal WS','B2B WS','B2C WS','Marketing Cost']].rename(columns={'Tool':'Name','Value':'Value $'})
t3_q['Type'] = 'Tool'

t_m_q = pd.concat([t_q,t2_q,t3_q])

t_f_q = t_m_q.groupby('Quarter - Year').sum().reset_index()
t_f_q

,Quarter - Year,Value $,MQL WS,SQL WS,OPP WS,Deal WS,B2B WS,B2C WS,Marketing Cost
0,2018 1,9490.32,0.0000,0.0000,0.0000,0.0000,4276.3200,5214.0000,5380.00
1,2018 2,18587.00,0.0000,0.0000,0.0000,0.0000,4527.0000,14060.0000,14400.00
2,2018 3,18377.00,0.0000,0.0000,0.0000,0.0000,3817.0000,14560.0000,14160.00
3,2018 4,8612.00,0.0000,0.0000,0.0000,0.0000,1572.0000,7040.0000,7000.00
4,2019 1,12976.00,11548.2500,379.2500,524.2500,524.2500,12976.0000,0.0000,21000.00
5,2019 2,8022.80,4477.2500,1419.6500,998.2500,998.2500,7893.4000,0.0000,6802.00
6,2019 3,13916.40,7911.0000,2013.2000,1802.0000,1802.0000,13528.2000,0.0000,13380.00
7,2019 4,23411.00,16499.5000,2730.5000,1862.0000,1862.0000,22954.0000,0.0000,29886.00
8,2020 1,251319.17,53754.3300,64683.1300,45858.0340,48949.1880,217521.6320,37477.4880,58339.38
9,2020 2,200288.88,40543.5300,48471.4500,40211.8600,40946.7700,170696.6100,29518.2700,38438.60


# CRM Information

## CRM - Monthly

In [18]:
!pip install -q simple_salesforce
# installing salesforce access and giving auth
import requests
consumer_key = "3MVG9p1Q1BCe9GmCjBcLl1WrzgMV_tAn1ES_O4voJ3S_nbtljEik0gqbgz7Im43Y5CmxgeBHU3GOPAeA_Or7u"
consumer_secret = "BB065F99EDDDFCC55B0AE8A65786FC186513F0BEB07C7FFE0E13E13A4A01060F"
redirect_url = "http://localhost/"
sf_user = "v.ramos@slangapp.com"
sf_pass = "Slangmonogatari17*"
auth_url = "https://login.salesforce.com/services/oauth2/token"

response = requests.post(auth_url, data= {'client_id':consumer_key,
                                          'client_secret':consumer_secret,
                                          'grant_type':'password',
                                          'username':sf_user,
                                          'password':sf_pass})
json_res = response.json()
access_token = json_res['access_token']
auth = {'Authorization':'Bearer ' + access_token}
instance_url = json_res['instance_url']

url = instance_url + '/services/data/v45.0/sobjects/contact/describe'
res = requests.get(url,headers=auth)
r = res.json()

# Accessing the API and creating a SOQL Function to transform json results into dataframe
from simple_salesforce import Salesforce
isSandbox = 0
sec_token = 'D0UBgLd33cgTRj8WI9FibbSy'
sf = Salesforce(username = sf_user,
                password = sf_pass,
                security_token = sec_token,
                instance_url = instance_url)

def SOQL(SOQL):
  qResult = sf.query(SOQL)
  print('Record Count {0}'.format(qResult['totalSize']))
  isDone = qResult['done'];
  if isDone == True:
    df = pd.DataFrame(qResult['records'])

  while isDone != True:
    try:
        if qResult['done'] != True:
          df = df.append(pd.DataFrame(qResult['records']));
          qResult = sf.query_more(qResult['nextRecordsUrl'], True)
        else:
          df = df.append(pd.DataFrame(qResult['records']))
          isDone = True;
          print('completed')
          break;
    except NameError:
      df = pd.DataFrame(qResult['records'])
      qry = sf.query_more(qResult['nextRecordsUrl'], True)
      
  return df.drop('attributes',axis=1);

#Creating the Dataframe for Outreach
sf_total = SOQL("SELECT ID, CC_MQL_DATE__C, C_MQL__C, CC_SQL_DATE__C, C_SQL__C, CC_OPPORTUNITY_DATE__C, C_OPP__C, CLOSEDATE, C_CLOSE__C, DEAL_OWNER_NAME__C, CC_DEAL_TYPE__C, STAGENAME FROM OPPORTUNITY WHERE ISDELETED = FALSE")

     |████████████████████████████████| 120 kB 4.8 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 214 kB 58.0 MB/s 
     |████████████████████████████████| 4.0 MB 59.2 MB/s 
     |████████████████████████████████| 41 kB 658 kB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
Record Count 15881
completed


In [19]:
sf_total[['CC_MQL_Date__c','CC_SQL_Date__c','CC_Opportunity_Date__c','CloseDate']] = sf_total[['CC_MQL_Date__c','CC_SQL_Date__c','CC_Opportunity_Date__c','CloseDate']].apply(pd.to_datetime)

mqls = sf_total[(sf_total['C_MQL__c']>0) 
                & (~sf_total['CC_Deal_Type__c'].isin(['Expansion','Renewal','Resurected','Pilot','Stand Alone'])) 
                  & (~sf_total['Deal_Owner_Name__c'].isin(['Juan Toro','Clouder Consulting','Vinicius Ramos','Maria Pardo','Juan Henao']))][['C_MQL__c','CC_MQL_Date__c']]
mqls = mqls.groupby(mqls['CC_MQL_Date__c'].dt.strftime('%Y-%m')).sum().reset_index()

sqls = sf_total[(sf_total['C_SQL__c']>0) 
                & (~sf_total['CC_Deal_Type__c'].isin(['Expansion','Renewal','Resurected','Pilot','Stand Alone'])) 
                  & (~sf_total['Deal_Owner_Name__c'].isin(['Juan Toro','Clouder Consulting','Vinicius Ramos','Maria Pardo','Juan Henao']))][['C_SQL__c','CC_SQL_Date__c']]
sqls = sqls.groupby(sqls['CC_SQL_Date__c'].dt.strftime('%Y-%m')).sum().reset_index()

opps = sf_total[(sf_total['C_OPP__c']>0) 
                & (~sf_total['CC_Deal_Type__c'].isin(['Expansion','Renewal','Resurected','Pilot','Stand Alone'])) 
                  & (~sf_total['Deal_Owner_Name__c'].isin(['Juan Toro','Clouder Consulting','Vinicius Ramos','Maria Pardo','Juan Henao']))][['C_OPP__c','CC_Opportunity_Date__c']]
opps = opps.groupby(opps['CC_Opportunity_Date__c'].dt.strftime('%Y-%m')).sum().reset_index()

new_booking = sf_total[(sf_total['C_Close__c']>0) 
                & (~sf_total['CC_Deal_Type__c'].isin(['Expansion','Renewal','Resurected'])) 
                  & (~sf_total['Deal_Owner_Name__c'].isin(['Juan Toro','Clouder Consulting','Vinicius Ramos','Maria Pardo','Juan Henao']))
                    & (sf_total['StageName'] == 'Closed Won')][['C_Close__c','CloseDate']]
new_booking = new_booking.groupby(new_booking['CloseDate'].dt.strftime('%Y-%m')).sum().reset_index()

funnel = mqls.merge(sqls,left_on='CC_MQL_Date__c',right_on='CC_SQL_Date__c',how='left') \
          .merge(opps,left_on='CC_MQL_Date__c',right_on='CC_Opportunity_Date__c',how='left') \
              .merge(new_booking,left_on='CC_MQL_Date__c',right_on='CloseDate',how='left') \
              [['CC_MQL_Date__c','C_MQL__c','C_SQL__c','C_OPP__c','C_Close__c']] \
              .rename(columns={'CC_MQL_Date__c':'Month - Year',
                                  'C_MQL__c':'MQL','C_SQL__c':'SQL','C_OPP__c':'OPP','C_Close__c':'Deal'})
funnel

,Month - Year,MQL,SQL,OPP,Deal
0,2020-01,166.0,67.0,26.0,2.0
1,2020-02,212.0,107.0,51.0,9.0
2,2020-03,146.0,84.0,45.0,4.0
3,2020-04,143.0,100.0,42.0,4.0
4,2020-05,147.0,96.0,53.0,7.0
5,2020-06,92.0,75.0,32.0,10.0
6,2020-07,238.0,70.0,33.0,9.0
7,2020-08,180.0,82.0,26.0,4.0
8,2020-09,178.0,77.0,31.0,9.0
9,2020-10,104.0,70.0,32.0,8.0


## CRM Quarterly

In [20]:
sf_total[['CC_MQL_Date__c','CC_SQL_Date__c','CC_Opportunity_Date__c','CloseDate']] = sf_total[['CC_MQL_Date__c','CC_SQL_Date__c','CC_Opportunity_Date__c','CloseDate']].apply(pd.to_datetime)

mqls_q = sf_total[(sf_total['C_MQL__c']>0) 
                & (~sf_total['CC_Deal_Type__c'].isin(['Expansion','Renewal','Resurected','Pilot','Stand Alone'])) 
                  & (~sf_total['Deal_Owner_Name__c'].isin(['Juan Toro','Clouder Consulting','Vinicius Ramos','Maria Pardo','Juan Henao']))][['C_MQL__c','CC_MQL_Date__c']]
mqls_q = mqls_q.groupby(mqls_q['CC_MQL_Date__c'].dt.year.astype(str) + ' ' + mqls_q['CC_MQL_Date__c'].dt.quarter.astype(str)).sum().reset_index()

sqls_q = sf_total[(sf_total['C_SQL__c']>0) 
                & (~sf_total['CC_Deal_Type__c'].isin(['Expansion','Renewal','Resurected','Pilot','Stand Alone'])) 
                  & (~sf_total['Deal_Owner_Name__c'].isin(['Juan Toro','Clouder Consulting','Vinicius Ramos','Maria Pardo','Juan Henao']))][['C_SQL__c','CC_SQL_Date__c']]
sqls_q = sqls_q.groupby(sqls_q['CC_SQL_Date__c'].dt.year.astype(str) + ' ' + sqls_q['CC_SQL_Date__c'].dt.quarter.astype(str)).sum().reset_index()

opps_q = sf_total[(sf_total['C_OPP__c']>0) 
                & (~sf_total['CC_Deal_Type__c'].isin(['Expansion','Renewal','Resurected','Pilot','Stand Alone'])) 
                  & (~sf_total['Deal_Owner_Name__c'].isin(['Juan Toro','Clouder Consulting','Vinicius Ramos','Maria Pardo','Juan Henao']))][['C_OPP__c','CC_Opportunity_Date__c']]
opps_q = opps_q.groupby(opps_q['CC_Opportunity_Date__c'].dt.year.astype(str) + ' ' + opps_q['CC_Opportunity_Date__c'].dt.quarter.astype(str)).sum().reset_index()

new_booking_q = sf_total[(sf_total['C_Close__c']>0) 
                & (~sf_total['CC_Deal_Type__c'].isin(['Expansion','Renewal','Resurected'])) 
                  & (~sf_total['Deal_Owner_Name__c'].isin(['Juan Toro','Clouder Consulting','Vinicius Ramos','Maria Pardo','Juan Henao']))
                    & (sf_total['StageName'] == 'Closed Won')][['C_Close__c','CloseDate']]
new_booking_q = new_booking_q.groupby(new_booking_q['CloseDate'].dt.year.astype(str) + ' ' + new_booking_q['CloseDate'].dt.quarter.astype(str)).sum().reset_index()

funnel_q = mqls_q.merge(sqls_q,left_on='CC_MQL_Date__c',right_on='CC_SQL_Date__c',how='left') \
          .merge(opps_q,left_on='CC_MQL_Date__c',right_on='CC_Opportunity_Date__c',how='left') \
              .merge(new_booking_q,left_on='CC_MQL_Date__c',right_on='CloseDate',how='left') \
              [['CC_MQL_Date__c','C_MQL__c','C_SQL__c','C_OPP__c','C_Close__c']] \
              .rename(columns={'CC_MQL_Date__c':'Quarter - Year',
                                  'C_MQL__c':'MQL','C_SQL__c':'SQL','C_OPP__c':'OPP','C_Close__c':'Deal'})
funnel_q

,Quarter - Year,MQL,SQL,OPP,Deal
0,2020 1,524.0,258.0,122.0,15.0
1,2020 2,382.0,271.0,127.0,21.0
2,2020 3,596.0,229.0,90.0,22.0
3,2020 4,303.0,186.0,81.0,19.0
4,2021 1,634.0,255.0,119.0,23.0
5,2021 2,839.0,438.0,167.0,29.0
6,2021 3,785.0,412.0,178.0,24.0
7,2021 4,727.0,396.0,177.0,40.0
8,2022 1,961.0,471.0,198.0,33.0
9,2022 2,734.0,355.0,135.0,31.0


# Model

## Monthly

In [34]:
g = t_f.merge(funnel, on= 'Month - Year',how='left')

g['MQL Cost'] = g['MQL WS']/g['MQL']
g['SQL Cost'] = g['SQL WS']/g['SQL']
g['OPP Cost'] = g['OPP WS']/g['OPP']
g['Deal Cost'] = g['Deal WS']/g['Deal']

g['Sales Cost'] = ((g['Deal WS'] + g['MQL WS'] + g['SQL WS'] + g['OPP WS']) - g['Marketing Cost'])
g['New Booking Cost'] = (g['Sales Cost'] + g['Marketing Cost'])/g['Deal']
g

,Month - Year,Value $,MQL WS,SQL WS,OPP WS,Deal WS,B2B WS,B2C WS,Marketing Cost,MQL,SQL,OPP,Deal,MQL Cost,SQL Cost,OPP Cost,Deal Cost,Sales Cost,New Booking Cost
0,2018-01,2779.44,0.0000,0.0000,0.0000,0.0000,1605.4400,1174.0000,1380.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1380.0000,NaN
1,2018-02,3932.44,0.0000,0.0000,0.0000,0.0000,1412.4400,2520.0000,2500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2500.0000,NaN
2,2018-03,2778.44,0.0000,0.0000,0.0000,0.0000,1258.4400,1520.0000,1500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1500.0000,NaN
3,2018-04,2859.00,0.0000,0.0000,0.0000,0.0000,1339.0000,1520.0000,1500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1500.0000,NaN
4,2018-05,4360.00,0.0000,0.0000,0.0000,0.0000,1340.0000,3020.0000,3000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3000.0000,NaN
5,2018-06,11368.00,0.0000,0.0000,0.0000,0.0000,1848.0000,9520.0000,9900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9900.0000,NaN
6,2018-07,8024.00,0.0000,0.0000,0.0000,0.0000,1504.0000,6520.0000,6160.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6160.0000,NaN
7,2018-08,5911.00,0.0000,0.0000,0.0000,0.0000,1391.0000,4520.0000,4500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4500.0000,NaN
8,2018-09,4442.00,0.0000,0.0000,0.0000,0.0000,922.0000,3520.0000,3500.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3500.0000,NaN
9,2018-10,3869.00,0.0000,0.0000,0.0000,0.0000,849.0000,3020.0000,3000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3000.0000,NaN


## Quarter

In [35]:
g_q = t_f_q.merge(funnel_q, on= 'Quarter - Year',how='left')

g_q['MQL Cost'] = g_q['MQL WS']/g['MQL']
g_q['SQL Cost'] = g_q['SQL WS']/g['SQL']
g_q['OPP Cost'] = g_q['OPP WS']/g['OPP']
g_q['Deal Cost'] = g_q['Deal WS']/g['Deal']

g_q['Sales Cost'] = ((g_q['Deal WS'] + g_q['MQL WS'] + g_q['SQL WS'] + g_q['OPP WS']) - g_q['Marketing Cost'])
g_q['New Booking Cost'] = (g_q['Sales Cost'] + g_q['Marketing Cost'])/g_q['Deal']
g_q

,Quarter - Year,Value $,MQL WS,SQL WS,OPP WS,Deal WS,B2B WS,B2C WS,Marketing Cost,MQL,SQL,OPP,Deal,MQL Cost,SQL Cost,OPP Cost,Deal Cost,Sales Cost,New Booking Cost
0,2018 1,9490.32,0.0000,0.0000,0.0000,0.0000,4276.3200,5214.0000,5380.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5380.0000,NaN
1,2018 2,18587.00,0.0000,0.0000,0.0000,0.0000,4527.0000,14060.0000,14400.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14400.0000,NaN
2,2018 3,18377.00,0.0000,0.0000,0.0000,0.0000,3817.0000,14560.0000,14160.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14160.0000,NaN
3,2018 4,8612.00,0.0000,0.0000,0.0000,0.0000,1572.0000,7040.0000,7000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7000.0000,NaN
4,2019 1,12976.00,11548.2500,379.2500,524.2500,524.2500,12976.0000,0.0000,21000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8024.0000,NaN
5,2019 2,8022.80,4477.2500,1419.6500,998.2500,998.2500,7893.4000,0.0000,6802.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1091.4000,NaN
6,2019 3,13916.40,7911.0000,2013.2000,1802.0000,1802.0000,13528.2000,0.0000,13380.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.2000,NaN
7,2019 4,23411.00,16499.5000,2730.5000,1862.0000,1862.0000,22954.0000,0.0000,29886.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6932.0000,NaN
8,2020 1,251319.17,53754.3300,64683.1300,45858.0340,48949.1880,217521.6320,37477.4880,58339.38,524.0,258.0,122.0,15.0,NaN,NaN,NaN,NaN,154905.3020,14216.312133
9,2020 2,200288.88,40543.5300,48471.4500,40211.8600,40946.7700,170696.6100,29518.2700,38438.60,382.0,271.0,127.0,21.0,NaN,NaN,NaN,NaN,131735.0100,8103.505238


# Viz

In [36]:
import plotly.express as px

fig = px.bar(g[pd.to_datetime(g['Month - Year']) >= '2020-01-01'], x='Month - Year', y='New Booking Cost',text_auto='.2s')
fig.update_layout(title_text='New Booking Cost by Month')
fig.show()     

In [37]:
fig = px.bar(g_q[pd.to_datetime(g_q['Quarter - Year']) >= '2020-01-01'], x='Quarter - Year', y='New Booking Cost',text_auto='.2s')
fig.update_layout(title_text='New Booking Cost by Quarter')
fig.show()    